#**Klasifikasi Lahan di Kawasan Ekonomi Khusus Perdagangan Bebas-Kota Batam menggunakan Support Vector Machine berbasis GEE**

## Authentikasi dan Insialisasi GEE

In [2]:
import ee
import geemap
import folium

#Authentikasi dan inisialisasi GEE
ee.Authenticate()
ee.Initialize(project='ee-bigdata121450146')

## ROI Kota Batam dan Koleksi Citra Sentinel-2 SR(Surface Reflectance)

In [4]:
# Inisialisasi peta
Map = geemap.Map(center=[1.1, 104], zoom=10)

#ROI Kota Batam
roi = ee.FeatureCollection("FAO/GAUL/2015/level2") \
  .filter(ee.Filter.eq('ADM2_NAME', 'Kota Batam')) \
  .filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))

Map.addLayer(roi, {}, "Kota Batam")

#Koleksi citra Sentinel-2 SR
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterBounds(roi) \
  .filterDate("2024-01-01", "2024-12-31") \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .map(lambda img: img.select(['B2', 'B3', 'B4', 'B8']).divide(10000).copyProperties(img, img.propertyNames()))

#Hitung median
median = collection.median().clip(roi)

#Tampilkan RGB
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
Map.addLayer(median, rgb_vis, "RGB")
Map

Map(center=[1.1, 104], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Membuat Training Data

In [6]:
#Inisialisasi peta
Map = geemap.Map(center=[1.1, 104], zoom=10)

#ROI Kota Batam
roi = ee.FeatureCollection("FAO/GAUL/2015/level2") \
  .filter(ee.Filter.eq('ADM2_NAME', 'Kota Batam')) \
  .filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))
Map.addLayer(roi, {}, "Kota Batam")

#Koleksi citra Sentinel-2 SR
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterBounds(roi) \
  .filterDate("2024-01-01", "2024-12-31") \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .map(lambda img: img.select(['B2', 'B3', 'B4', 'B8']).divide(10000).copyProperties(img, img.propertyNames()))
#Hitung median
median = collection.median().clip(roi)

#Tampilkan RGB
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
Map.addLayer(median, rgb_vis, "RGB")

#Membuat Training data
#Polygon training manual
areaterbangun = ee.Geometry.Polygon(
    [[[104.02539263565725,1.1400776826761825],
      [104.03054247696585,1.136988387915249],
      [104.03346072037405,1.1491739201887468],
      [104.0248776515264,1.1481441589012948],
      [104.02539263565725,1.1400776826761825]]])

vegetasi = ee.Geometry.Polygon(
    [[[103.97358614707248,1.0838786625085348],
      [103.98036677146213,1.0734091740699037],
      [103.98663241172092,1.080274416457961],
      [103.97736269736545,1.0873112738069546],
      [103.97358614707248,1.0838786625085348]]])

lahanterbuka = ee.Geometry.Polygon(
    [[[104.1054278782567,1.1480566869205289],
      [104.12576975142565,1.1458255361241416],
      [104.12036241805163,1.153205489799842],
      [104.10680116927233,1.153205489799842],
      [104.1054278782567,1.1480566869205289]]])

#fitur dan kelas
builtupFeature = ee.Feature(areaterbangun, {'class': 0})
vegetationFeature = ee.Feature(vegetasi, {'class': 1})
barelandFeature = ee.Feature(lahanterbuka, {'class': 2})

trainingPolygons = ee.FeatureCollection([builtupFeature, vegetationFeature, barelandFeature])
Map.addLayer(trainingPolygons, {}, 'Training Areas')
Map

Map(center=[1.1, 104], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Model SVM dan hasil klasifikasi

In [8]:
#Inisialisasi peta
Map = geemap.Map(center=[1.1, 104], zoom=10)

#ROI Kota Batam
roi = ee.FeatureCollection("FAO/GAUL/2015/level2") \
  .filter(ee.Filter.eq('ADM2_NAME', 'Kota Batam')) \
  .filter(ee.Filter.eq('ADM0_NAME', 'Indonesia'))

Map.addLayer(roi, {}, "Kota Batam")

#Koleksi citra Sentinel-2 SR
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterBounds(roi) \
  .filterDate("2024-01-01", "2024-12-31") \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .map(lambda img: img.select(['B2', 'B3', 'B4', 'B8']).divide(10000).copyProperties(img, img.propertyNames()))

#Hitung median
median = collection.median().clip(roi)

#Tampilkan RGB
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
Map.addLayer(median, rgb_vis, "RGB")

#Polygon training manual
areaterbangun = ee.Geometry.Polygon(
    [[[104.02539263565725,1.1400776826761825],
      [104.03054247696585,1.136988387915249],
      [104.03346072037405,1.1491739201887468],
      [104.0248776515264,1.1481441589012948],
      [104.02539263565725,1.1400776826761825]]])

vegetasi = ee.Geometry.Polygon(
    [[[103.97358614707248,1.0838786625085348],
      [103.98036677146213,1.0734091740699037],
      [103.98663241172092,1.080274416457961],
      [103.97736269736545,1.0873112738069546],
      [103.97358614707248,1.0838786625085348]]])

lahanterbuka = ee.Geometry.Polygon(
    [[[104.1054278782567,1.1480566869205289],
      [104.12576975142565,1.1458255361241416],
      [104.12036241805163,1.153205489799842],
      [104.10680116927233,1.153205489799842],
      [104.1054278782567,1.1480566869205289]]])

#fitur dan kelas
builtupFeature = ee.Feature(areaterbangun, {'class': 0})
vegetationFeature = ee.Feature(vegetasi, {'class': 1})
barelandFeature = ee.Feature(lahanterbuka, {'class': 2})

trainingPolygons = ee.FeatureCollection([builtupFeature, vegetationFeature, barelandFeature])
Map.addLayer(trainingPolygons, {}, 'Training Areas')

#Sampling
training = median.sampleRegions(
    collection=trainingPolygons,
    properties=['class'],
    scale=10
)

#Split Data
withRandom = training.randomColumn('random')
split = 0.7
trainingSet = withRandom.filter(ee.Filter.lt('random', split))
testingSet = withRandom.filter(ee.Filter.gte('random', split))

#Klasifikasi SVM
classifier = ee.Classifier.libsvm().train(
    features=trainingSet,
    classProperty='class',
    inputProperties=['B2', 'B3', 'B4', 'B8']
)

classified = median.classify(classifier)

#Visualisasi klasifikasi
palette = ['black', 'green', 'white'] #
Map.addLayer(classified, {'min': 0, 'max': 2, 'palette': palette}, 'Klasifikasi SVM')
Map


Map(center=[1.1, 104], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### Area berwarna: Hitam(Area terbangun-kelas 0), Hijau(Vegetasi-kelas 1), Putih(Lahan terbuka-kelas 1)

## Evaluasi Akurasi

In [10]:
valid = testingSet.classify(classifier)
conf_matrix = valid.errorMatrix('class', 'classification')

print('Confusion Matrix (SVM):')
print(conf_matrix.getInfo())
print('Overall Accuracy (SVM):', conf_matrix.accuracy().getInfo())


Confusion Matrix (SVM):
[[2535, 20, 98], [0, 3146, 0], [156, 0, 3687]]
Overall Accuracy (SVM): 0.9715826591993363
